In [1]:
import os
import json
import logging 

import openai
from dotenv import load_dotenv
import DuffelManager

load_dotenv(".env")
openai.api_key = os.getenv("OPENAI_API_KEY")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [2]:
functions =  [
  {
    "name": "book_best_flight",
    "description": "Will return confirmation for a flight booked given a departure city, arrival city, departure date",
    "parameters": {
      "type": "object",
      "properties": {
        "departure_city": {
          "type": "string",
          "description": "The 3 letter IATA code for the airport in the city of departure e.g. JFK, LHR, CDG"
        },
        "destination_city": {
          "type": "string",
          "description": "The 3 letter IATA code for the airport in the city of arrival e.g. JFK, LHR, CDG"
        },
        "departure_date": {
          "type": "string",
          "description": "The date of departure in the format YYYY-MM-DD"
        },
        "time_of_day": {
          "type": "string",
          "enum": ["morning", "afternoon", "evening", "night"]
        },
        "airline_name": {
          "type": "string",
          "description": "The 2 letter IATA code for the airline the user would like to fly in e.g. DL, UA, AA"
        },
        "cabin_class": {
          "type": "string",
          "enum": ["first", "business", "premium_economy", "economy"]
        },
      },
      "required": ["departure_city", "destination_city", "departure_date"]
    }
  },
  {
    "name": "get_absolute_date",
    "description": "Converts relative dates to absolute dates (only use if you can't convert the query into a date on your own)",
    "parameters": {
      "type": "object",
      "properties": {
        "relative_date": {
          "type": "string",
          "description": "The relative date to convert e.g. tomorrow, next week, next month"
        }
      },
      "required": ["relative_date"]
    }
  }
]

In [10]:
messages = [
  {"role": "system", "content": "You're a friendly flight booking assistant. You will be given a user query. Do your best to parse the user's request and respond accordingly. If the user hasn't given you enough information to make a function call, ask them for clarification"}
]

In [11]:
def call_gpt(chat_history: list[dict]):
  return openai.chat.completions.create(
    model="gpt-4-1106-preview",
    messages=chat_history,
    functions=functions,
    function_call="auto"
  )

def user_interaction(query: str): 
  reason = "function_call"
  messages.append({"role": "user", "content": query})
  
  #allows gpt to continue calling functions until it's ready to return to the user
  while reason == "function_call":
    response = call_gpt(messages)
    logging.info(f"response: {response}")
    reason = response.choices[0].finish_reason
    if reason == "function_call":
      kwargs = json.loads(response.choices[0].message.function_call.arguments)
      function_to_call = getattr(DuffelManager, response.choices[0].message.function_call.name)
      messages.append({"role": "assistant", "content": "sure i'm happy to help"})
      logging.info(f"making {function_to_call.__name__} call with arguments: {kwargs}")
      
      #make the function call and add response to messages
      response_obj = function_to_call(**kwargs)
      if response_obj.success:
        messages.append({"role": "function", "content": str(response_obj.resp), "name": function_to_call.__name__})
      else:
        messages.append({"role": "function", "content": str(response_obj.error), "name": function_to_call.__name__})
    else: # ready to return to user
      messages.append({"role": "assistant", "content": response.choices[0].message.content})
      return messages[-1]["content"]

In [12]:
messages

[{'role': 'system',
  'content': "You're a friendly flight booking assistant. You will be given a user query. Do your best to parse the user's request and respond accordingly. If the user hasn't given you enough information to make a function call, ask them for clarification"}]

In [13]:
user_interaction("I'd like to book a flight from New york to san francisco tomorrow morning")

2023-11-10 17:57:14,186 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-10 17:57:14,187 - root - INFO - response: ChatCompletion(id='chatcmpl-8JUj3J5y5whxq4xj8WGW5hYE1ZSTW', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"relative_date":"tomorrow"}', name='get_absolute_date'), tool_calls=None))], created=1699657033, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_a24b4d720c', usage=CompletionUsage(completion_tokens=17, prompt_tokens=335, total_tokens=352))
2023-11-10 17:57:14,188 - root - INFO - making get_absolute_date call with arguments: {'relative_date': 'tomorrow'}
2023-11-10 17:57:15,613 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-10 17:57:15,615 - root - INFO - response: ChatCompletion(id='chatcmpl-8JUj4ti0qfWY4MpxJ2pOGj9nOgKg

"Your flight from New York to San Francisco has been booked for tomorrow morning, November 11th, 2023. You'll be flying with American Airlines on a non-stop flight, departing at 07:44 AM in economy class. The total for the ticket is $211.54. Safe travels!"

In [14]:
user_interaction("Actually I'd like to take the flight next tuesday please.") 

2023-11-10 17:57:39,580 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-10 17:57:39,583 - root - INFO - response: ChatCompletion(id='chatcmpl-8JUjSYkFRbFbgGC8c0wHPIDkyyWDZ', choices=[Choice(finish_reason='function_call', index=0, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"relative_date":"next tuesday"}', name='get_absolute_date'), tool_calls=None))], created=1699657058, model='gpt-4-1106-preview', object='chat.completion', system_fingerprint='fp_a24b4d720c', usage=CompletionUsage(completion_tokens=18, prompt_tokens=488, total_tokens=506))
2023-11-10 17:57:39,583 - root - INFO - making get_absolute_date call with arguments: {'relative_date': 'next tuesday'}
2023-11-10 17:57:41,259 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-11-10 17:57:41,261 - root - INFO - response: ChatCompletion(id='chatcmpl-8JUjT6PXaVTFXX86PWxT

'Your flight from New York to San Francisco has been rebooked for next Tuesday, November 14th, 2023. You will depart at 10:03 AM on a non-stop flight with British Airways in economy class. The cost of the ticket is $208.55. Is there anything else I can assist you with?'

In [ ]:
### things to improve
  # 1. Considered caching requests in case similar requests were made. Might not be worth it since offers change so frequently
  # 2. I haven't made any IATA validators, so airline and flight may not always be correct, but can easily be implemented using the gpt function calling
  # 3. I'd like to expand the success and failure response to include both types of responses (datetime and MyOffer classes) instead of a generic catchall obj
  # 4. Implement error handling for a query returning no flights (didn't run across this issue in my numerous testing)

### things to in best offer function
  # 1. Instead of giving up on time of day if there are none at that time, we should try to find the closest time
  # 2. Same idea with cabin class, if there are no offers for the specified cabin class, we should try to find the closest cabin class
  
# i'm not sure what the best practice is for appending a function call back to the message history, the model decides to format the output in a non function call format otherwise. I'd love to hear any alternatives